# ABS Monthly Household Spending Indicator 5682

## Python set-up

In [1]:
# analytic imports
import matplotlib.pyplot as plt
import pandas as pd
import readabs as ra
from readabs import metacol as mc

# local imports
from abs_helper import get_abs_data
from plotting import clear_chart_dir, plot_growth_finalise, set_chart_dir

# display charts in this notebook
SHOW = False

In [2]:
abs_dict, meta, source, RECENT = get_abs_data("5682.0")
plot_times = None, RECENT
plot_tags = "", "-recent"

## Plotting

### Headline growth

In [3]:
def headline_charts() -> None:
    """Produce headline charts."""

    long_title = 20
    data_sets = {
        "5682001_National": "Monthly",
        # "1Q": "Quarterly",   # not available???
    }
    for table, period in data_sets.items():
        if table not in abs_dict:
            continue
        data = abs_dict[table]

        adjustment = "Calendar adjusted"
        prices = "Current Price"
        series_type = "Original"
        base_selector = {
            table: mc.table,
            series_type: mc.stype,
            "household spending": mc.did,
            adjustment: mc.did,
            prices: mc.did,
        }
        annual_selector = base_selector | {
            "Through the year percentage change": mc.did,
        }
        annual_rows = ra.search_abs_meta(meta, annual_selector, verbose=False)
        period_selector = base_selector | {
            f"{period} percentage change": mc.did,
        }
        period_rows = ra.search_abs_meta(meta, period_selector, verbose=False)

        for chart in [x.split(";")[1].strip() for x in annual_rows[mc.did]]:
            print(chart)

            # data wrangling
            annual_id = annual_rows[
                annual_rows[mc.did].str.contains(chart, regex=False)
            ][mc.id].iloc[0]
            period_id = period_rows[
                period_rows[mc.did].str.contains(chart, regex=False)
            ][mc.id].iloc[0]

            if annual_id not in data.columns or period_id not in data.columns:
                print(
                    f"Bailing on {chart} where annual={annual_id} and period={period_id}"
                )
                continue

            annual, periodic = data[annual_id], data[period_id]

            # plotting
            separator = " " if len(chart) <= long_title else "\n"
            plot_growth_finalise(
                annual,
                periodic,
                title=f"Growth in Household Spending:{separator}{chart}",
                ylabel="Per cent",
                lfooter=f"Australia. {series_type} series. {prices}. {adjustment}. ",
                rfooter=source,
                show=SHOW,
            )


headline_charts()

Food
Alcoholic beverages and tobacco
Clothing and footwear
Furnishings and household equipment
Health
Transport
Recreation and culture
Hotels, cafes and restaurants
Miscellaneous goods and services
Goods
Services
Discretionary
Non-discretionary
Total (Household Spending Categories)


## Finished

In [4]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Thu Aug 01 2024 13:32:37

Python implementation: CPython
Python version       : 3.12.4
IPython version      : 8.26.0

pandas    : 2.2.2
readabs   : 0.0.17
matplotlib: 3.9.1

Watermark: 2.4.3



In [5]:
print("Done")

Done
